In [1]:
import os
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchnet import meter
from tqdm import tqdm
import numpy as np
import time
import torch.nn.functional as F
import torch.nn as nn


In [2]:
from cqt_loader_setlist import CQT
from models import CQTNet
from config import DefaultConfig
from utility import calc_MAP, norm

# Set Parameters

In [3]:
torch.cuda.is_available()

True

In [25]:
modelname = 'CQTNet'
load_latest = True
load_model_path = 'check_points/CQTNet.pth'
model_save_path = 'finetuning'

load_latest=True 

parallel=True

opt = DefaultConfig(load_model_path=load_model_path, 
                    notes=model_save_path,
                    batch_sz=32,
                    max_epoch=50, 
                    lr=0.001, 
                    lr_decay=0.8, 
                    weight_decay=1e-5, 
                    use_gpu=True, 
                    num_workers=4, 
                    parallel=parallel, 
                    load_latest=load_latest,
                    gpu_device="cuda:0")


torch.cuda.is_available() True
use_gpu True
cuda:0


In [5]:
torch.cuda.get_device_name(device=0)

'NVIDIA GeForce RTX 3090'

In [6]:
#root = "/Users/dirceusilva/Documents/BancoDados/setlist_all/setlist_65k/features_develop/universe_develop"
root = '/mnt/dataset/dataset_cover/universe_develop'
train_data_path = os.path.join(root, "universe_train")
val_data_path = os.path.join(root, "universe_val")
test_data_path = os.path.join(root, "universe_test")

# step 1: Build DataLoader

In [7]:
#######################################################
#                  Create Dataset
#######################################################
train_dataset0 = CQT(train_data_path, mode="train", out_length=200, num_workers=opt.num_workers)
train_dataset1 = CQT(train_data_path, mode="train", out_length=300, num_workers=opt.num_workers)
train_dataset2 = CQT(train_data_path, mode="train", out_length=400, num_workers=opt.num_workers)

valid_dataset = CQT(val_data_path, mode="valid", out_length=None, num_workers=1) 
test_dataset = CQT(test_data_path, mode="test", out_length=None, num_workers=1)

#######################################################
#                  Define Dataloaders
#######################################################

train_dataloader0 = DataLoader(train_dataset0, batch_size=opt.batch_size, shuffle=True, generator=torch.Generator(device='cuda:0'))
train_dataloader1 = DataLoader(train_dataset1, batch_size=opt.batch_size, shuffle=True, generator=torch.Generator(device='cuda:0'))
train_dataloader2 = DataLoader(train_dataset2, batch_size=opt.batch_size, shuffle=True, generator=torch.Generator(device='cuda:0'))

val_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=False, generator=torch.Generator(device='cuda:0'))
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, generator=torch.Generator(device='cuda:0'))

loaders = {'train': train_dataloader0, 'valid': val_dataloader, 'test': test_dataloader}


# step 2: configure model

In [15]:

#opt._parse(kwargs)

model = CQTNet.CQTNet()
model.load_state_dict(torch.load(load_model_path, map_location=opt.device))

print(model)

CQTNet(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3)

In [16]:
model = CQTNet.CQTNetSetlist(original_model=model, num_classes=train_dataset0.get_nr_classes())

In [17]:
for p in model.features.parameters():
    p.requires_grad = True

for p in model.fc0.parameters():
    p.requires_grad = True

In [18]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

features.conv0.weight True
features.norm0.weight True
features.norm0.bias True
features.conv1.weight True
features.norm1.weight True
features.norm1.bias True
features.conv2.weight True
features.norm2.weight True
features.norm2.bias True
features.conv3.weight True
features.norm3.weight True
features.norm3.bias True
features.conv4.weight True
features.norm4.weight True
features.norm4.bias True
features.conv5.weight True
features.norm5.weight True
features.norm5.bias True
features.conv6.weight True
features.norm6.weight True
features.norm6.bias True
features.conv7.weight True
features.norm7.weight True
features.norm7.bias True
features.conv8.weight True
features.norm8.weight True
features.norm8.bias True
features.conv9.weight True
features.norm9.weight True
features.norm9.bias True
fc0.weight True
fc0.bias True
fc1.weight True
fc1.bias True


In [19]:
if parallel is True: 
        model = torch.nn.DataParallel(model, device_ids=[0])

if parallel is True:
    if opt.load_latest is True:
        #model.load_state_dict(torch.load(os.path.join("check_points",opt.notes)), strict=False)
        model.module.load_latest(opt.notes)
    elif opt.load_model_path:
        model.module.load(opt.load_model_path)
else:
    if opt.load_latest is True:
        model.load_latest(opt.notes)
    elif opt.load_model_path:
        model.load(opt.load_model_path)
model.to(opt.device)

check_points/<class 'models.CQTNet.CQTNetSetlist'>finetuning/latest.pth


DataParallel(
  (module): CQTNetSetlist(
    (features): Sequential(
      (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
      (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
      (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

# To device

In [20]:
model.to(opt.device)
print(model)

DataParallel(
  (module): CQTNetSetlist(
    (features): Sequential(
      (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
      (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
      (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
      (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

#  step 3: criterion and optimizer

In [21]:
criterion = torch.nn.CrossEntropyLoss()
lr = opt.lr
#if opt.parallel is True:
#    optimizer = torch.optim.Adam(model.module.parameters(), lr=lr, weight_decay=opt.weight_decay)
#else:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=opt.weight_decay)
    
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=opt.lr_decay, patience=2, min_lr=5e-6)

# step 4: Train

### Validation Function

In [22]:
@torch.no_grad()
def val_slow(model, dataloader, epoch):
    
    model.eval()
    total, correct = 0, 0
    labels, features = None, None

    for ii, (data, label) in enumerate(dataloader):
        input_data = data.to(opt.device)
        #print(input.shape)
        score, feature = model(input_data)
        feature = feature.data.cpu().numpy()
        label = label.data.cpu().numpy()
        if features is not None:
            features = np.concatenate((features, feature), axis=0)
            labels = np.concatenate((labels,label))
        else:
            features = feature
            labels = label
    features = norm(features)

    dis2d = -np.matmul(features, features.T) # [-1,1] Because normalized, so mutmul is equal to ED
    np.save('dis.npy',dis2d)
    np.save('label.npy',labels)

    MAP, top10, rank1 = calc_MAP(dis2d, labels)

    print(epoch, MAP, top10, rank1 )
    
    model.train()
    
    return MAP

### Training

In [23]:
best_MAP=0
    
val_slow(model, val_dataloader, -1)

-1 0.885741948128748 0.1830084235860409 33.35643802647413


0.885741948128748

In [26]:

for epoch in range(opt.max_epoch):
    running_loss = 0
    num = 0
    for (data0, label0),(data1, label1),(data2, label2) in tqdm(zip(train_dataloader0, train_dataloader1, train_dataloader2)):
        for flag in range(3):
            if flag==0:
                data=data0
                label=label0
            elif flag==1:
                data=data1
                label=label1
            else:
                data=data2
                label=label2
            # train model
            input_data = data.requires_grad_()
            input_data = input_data.to(opt.device)
            
            target = label.to(opt.device)

            optimizer.zero_grad()
            score, _ = model(input_data)
            loss = criterion(score, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num += target.shape[0]
        
    running_loss /= num 
    
    print(running_loss)
    
    if opt.parallel is True:
        model.module.save(opt.notes)
    else:
        model.save(opt.notes)
    
    # update learning rate
    scheduler.step(running_loss) 
    
    # validate
    MAP=0
    MAP += val_slow(model, val_dataloader, epoch)
            
    if MAP>best_MAP:
        best_MAP=MAP
        print('*****************BEST*****************')
    print('')
    model.train()

0it [00:00, ?it/s]

1273it [14:32,  1.46it/s]


0.04845547107165218
model name 0515_23:55:06.pth
0 0.8959867525726601 0.18456077015643804 37.63790613718412
*****************BEST*****************



1273it [14:30,  1.46it/s]


0.021686241091678635
model name 0516_00:11:34.pth
1 0.8945765534066048 0.18400722021660648 34.38688327316486



...

0.004528028591133716
model name 0516_12:49:17.pth
47 0.8946125950218352 0.18452466907340553 33.02635379061372



1273it [14:29,  1.46it/s]


0.004587180139367258
model name 0516_13:05:44.pth
48 0.8979962095564893 0.18523465703971118 32.58146811070999
*****************BEST*****************



1273it [14:28,  1.47it/s]


0.003742056920366177
model name 0516_13:22:10.pth
49 0.8921344339553335 0.18422382671480145 34.11347773766546



In [35]:
opt.notes='experiment0'

# step 5: Test

In [ ]:
def test(**kwargs):
    opt.batch_size=1
    opt.num_workers=1
    opt.model = 'CQTNet'
    opt.load_latest = False
    opt.load_model_path = 'check_points/CQTNet.pth'
    opt._parse(kwargs)
    
    model = getattr(models, opt.model)() 
    #print(model)
    if opt.load_latest is True:
        model.load_latest(opt.notes)
    elif opt.load_model_path:
        model.load(opt.load_model_path)
    model.to(opt.device)

    val_data = CQT('val', out_length=None)
    test_data = CQT('test', out_length=None)
    val_dataloader = DataLoader(val_data, 1, shuffle=False,num_workers=1)
    test_dataloader = DataLoader(test_data, 1, shuffle=False,num_workers=1)
    
    val_slow(model, val_dataloader, 0)

In [40]:
val_slow(model, test_dataloader, -1)

-1 0.900768927377684 0.0933318161128812 111.79449248975877


0.900768927377684